In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(Seurat)
    library(Signac)
    library(EnsDb.Hsapiens.v86)
    library(dplyr)
    library(ggplot2)
    library(bedr)
    library(SeuratDisk)
    library(scales)
    library(reshape2)
    library(Hmisc)
    library(tidyr)
    library(tidyverse)
    library(crayon)
    library(SeuratData)
    library(CelliD)
    library(readr)
}))

# Definitions etc

In [2]:
set.seed(1234)
options(digits=2)
stats <- c()

In [3]:
if(!exists("papermill_run")) {
    prj_name = "Screen1_66guides"
    data_subdir_suffixes = "L1,L2,L3,L4"     
    secondary_a_path = "/home/vsevim/prj/1012-ckd/S1/analysis/secondary/"
    save_seurat_h5 = "YES"
}

In [4]:
custom_theme <- theme(
  plot.title = element_text(size=16, hjust = 0.5), 
  legend.key.size = unit(0.7, "cm"), 
  legend.text = element_text(size = 14))

### Load Seurat file

In [5]:
integrated_h5_path <- "/home/vsevim/prj/1012-ckd/S1/analysis/secondary/integrated/seurat_objects/integrated.h5seurat"
seurat_combined    <- LoadH5Seurat(integrated_h5_path, verbose = FALSE)

Validating h5Seurat file

Warning message:
"Adding a command log without an assay associated with it"


# Differential expression

Split construct+/- cells

In [7]:
head(seurat_combined)

,nCount_RNA,nFeature_RNA,nCount_sgRNA,nFeature_sgRNA,nCount_SCT,nFeature_SCT,orig.ident,library,percent.mt,PS_call,construct_call,target_call,nCalled_Constructs,nPS_calls,nCalled_sgRNA_UMIs,status,donor,SCT_snn_res.0.5,seurat_clusters
,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<fct>,<fct>
AGCTCAATCGTAGGGA-1_1,32644,5712,92,9,28010,5671,Screen1_66guides,Lib_1,4.8,C15orf40_1a,C15orf40_1,C15orf40,1,1,57,singlet,3,2,2
CAACAGTTCCTAGCGG-1_1,40788,6939,1412,23,27919,6655,Screen1_66guides,Lib_1,6.4,C15orf40_1a,C15orf40_1,C15orf40,1,1,1383,doublet,1/2,0,0
AAACGCTCAGTTAGAA-1_1,44827,7247,3081,27,27830,6556,Screen1_66guides,Lib_1,9.7,C15orf40_1a|C15orf40_1b,C15orf40_1,C15orf40,1,2,1302|1753,singlet,0,3,3
AAAGGATAGCGATTCT-1_1,50419,7705,2006,14,28153,6639,Screen1_66guides,Lib_1,9.5,C15orf40_1a|C15orf40_1b,C15orf40_1,C15orf40,1,2,428|1565,singlet,0,2,2
ATCGGCGGTCTAGATC-1_1,40213,6389,4873,32,27819,6124,Screen1_66guides,Lib_1,6.9,C15orf40_1a|C15orf40_1b,C15orf40_1,C15orf40,1,2,1515|3322,singlet,2,2,2
ATTCCCGCATCCGCGA-1_1,30000,6331,2212,15,27913,6262,Screen1_66guides,Lib_1,9.7,C15orf40_1a|C15orf40_1b,C15orf40_1,C15orf40,1,2,606|1592,doublet,1/2,3,3
CAAGAGGCAGTTCTAG-1_1,32899,5995,783,9,28043,5935,Screen1_66guides,Lib_1,5.6,C15orf40_1a|C15orf40_1b,C15orf40_1,C15orf40,1,2,343|433,singlet,1,3,3
CCTGCATTCAAGCCAT-1_1,39516,7020,4468,87,27997,6802,Screen1_66guides,Lib_1,12.0,C15orf40_1a|C15orf40_1b,C15orf40_1,C15orf40,1,2,1943|2353,singlet,1,3,3
GTAATCGCACATACGT-1_1,72098,9371,2916,23,27635,6632,Screen1_66guides,Lib_1,7.1,C15orf40_1a|C15orf40_1b,C15orf40_1,C15orf40,1,2,1337|1554,singlet,1,1,1


In [10]:
seurat_combined$sgRNA

Assay data with 132 features for 24563 cells
First 10 features:
 CD13-1a, CD13-1b, CD55-1a, CD55-1b, CD73-1a, CD73-1b, FUBP1-1a,
FUBP1-1b, C15orf40-1a, C15orf40-1b 